# Mood Prediction Using Smartphone Data
### Data Mining Techniques | Vrije Universiteit Amsterdam | April 2024
##### Lieve Jilesen (ljn278), Ryan Ott (rot280), and Jaime Perez y Perez (jpz240) | Group 17

## 1 - Data Preparation

### 1.1 - Exploratory Data Analysis

#### Importing packages & loading data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import numpy as np
# sns.set(style="whitegrid")
# import statsmodels.api as sm

file_path = 'data/dataset_mood_smartphone.csv'
df = pd.read_csv(file_path, parse_dates=['time'], index_col=0)
df

#### General data properties

In [ ]:
num_records = df.shape[0]
num_participants = df.id.nunique()
num_days = df.time.dt.date.nunique()
num_variables = df.variable.nunique()

data_characteristics = {
    "Number of Records": [num_records],
    "Number of Participants": [num_participants],
    "Number of Days": [num_days],
    "Number of Variables": [num_variables],
}

data_characteristics_df = pd.DataFrame(data_characteristics)
data_characteristics_df

#### Variable properties

In [ ]:
# Create a summary table for the dataset
summary_data = {
    "Variable": [],
    "Data Type": [],
    "Total Records": [],
    "Unique Values": [],
    "Missing Values": [],
    "Mean": [],
    "Std": [],
    "Min": [],
    "25%": [],
    "50%": [],
    "75%": [],
    "Max": []
}

for var in df.variable.unique():
    var_data = df[df['variable'] == var]['value']
    desc = var_data.describe()
    summary_data["Variable"].append(var)
    summary_data["Data Type"].append(df[df['variable'] == var]['value'].dtype)
    summary_data["Total Records"].append(var_data.count())
    summary_data["Unique Values"].append(var_data.nunique())
    summary_data["Missing Values"].append(df[df['variable'] == var]['value'].isnull().sum())
    summary_data["Mean"].append(desc['mean'])
    summary_data["Std"].append(desc['std'])
    summary_data["Min"].append(desc['min'])
    summary_data["25%"].append(desc['25%'])
    summary_data["50%"].append(desc['50%'])
    summary_data["75%"].append(desc['75%'])
    summary_data["Max"].append(desc['max'])

summary_table = pd.DataFrame(summary_data)
summary_table


#### Any records missing id or time?

In [ ]:
# Do we have any records without index, id or time?
df[df['id'].isnull() | df['time'].isnull()]

No, thankfully not.

#### Distribution of variable values

In [ ]:
variables = [var for var in df['variable'].unique() if var not in ['sms', 'call']]
fig, axes = plt.subplots(nrows=len(variables), ncols=1, figsize=(10, 6 * len(variables)))

for ax, var in zip(axes.flatten(), variables):
    var_data = df[df['variable'] == var]['value'].dropna()
    mean = var_data.mean()
    std = var_data.std()
    # Adjust bins for better visualization based on data range and characteristics
    bins = min(30, int(var_data.nunique()))  # Use a minimum of 30 bins or less if fewer unique values

    ax.hist(var_data, bins=bins, alpha=0.75, color='blue', edgecolor='black', label=f'{var} Scores')
    ax.set_title(f'Distribution of {var} values')
    ax.set_xlabel(f'{var.capitalize()} Score')
    ax.set_ylabel('Frequency')
    ax.grid(axis='y', alpha=0.75)
    legend_label = f"Mean: {mean:.2f}, Std: {std:.2f}"
    ax.legend([f"{var.capitalize()} Scores\n{legend_label}"], loc='upper right', title='Statistics', frameon=True)

plt.tight_layout()
plt.show()

Histogram showing the distribution of possible mood scores in our dataset. The mood of participants seems to be normally distributed with a mean and standard deviation of around 7 and 1 respectively.

#### Mood for a participant

In [ ]:
# Time Series of Mood for a Selected Participant
participant_data = df[(df['id'] == 'AS14.01') & (df['variable'] == 'mood')]
plt.figure(figsize=(14, 7))
plt.plot(participant_data['time'], participant_data['value'], marker='o', linestyle='-')
plt.title('Mood Over Time for Participant AS14.01')
plt.xlabel('Time')
plt.ylabel('Mood Score')
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()

Timeseries plot of the mood scores given by the first participant. We see it being around 7 +- 1 for most of the time, with rare moments of extreme emotions. We also see long gaps in the record availability.

#### Correlation map for variables

In [ ]:
# Correlation heatmap of variables
pivot_table = df.pivot_table(index=['id', 'time'], columns='variable', values='value')
correlation_matrix = pivot_table.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Heatmap of Variables')
plt.show()

Correlation heatmap to visualize the correlation between different variables in the dataset. The heatmap provides a color-coded representation of the correlation values, with warmer colors indicating stronger positive correlations and cooler colors indicating stronger negative correlations. (Call is not visible as its boolean and only recorded when a call takes place so it always has a value of 1).

Arousal, and especially valence seem to be highly positively corellated with mood for example, while using finance apps seems to highly negatively correlate with screen time, which could be useful features to consider when making new predictions.

#### Activity & mood relationship

In [ ]:
# Activity and Mood Relationship
activity_data = df[df['variable'] == 'activity']
mood_data = df[df['variable'] == 'mood']

# Merging on ID and Time for direct comparison
activity_mood_merged = pd.merge(activity_data, mood_data, on=['id', 'time'], suffixes=('_activity', '_mood'))

plt.figure(figsize=(10, 6))
plt.scatter(activity_mood_merged['value_mood'], activity_mood_merged['value_activity'], alpha=0.5)
plt.title('Mood vs. Activity')
plt.xlabel('Mood Score')
plt.ylabel('Activity Score')
plt.xticks(range(1, 11))
plt.grid(axis='y', alpha=0.75)
plt.show()

Plot showing co-occurence of mood with activiy scores. If the mood is in the common range of 7±1 we see participants record a wide range of activity scores, while if the mood is low, there is no high activity. 

### 1.2 - Data Cleaning (Still w.i.p. ryan)

#### Adapting datatypes

Because call and sms are boolean (only present if true) we convert them to boolean values.

In [ ]:
# Convert 'call' and 'sms' data to boolean
for variable in ['call', 'sms']:
    df.loc[df['variable'] == variable, 'value'] = df[df['variable'] == variable]['value'].fillna(0).astype(bool)

#### Removing incorrect values

Because all `appCat.*` variables and `screen` should denote a time duration, they cannot be negative. Thus, any records with negative duration must be incorrect and removed. We see this is the case for example when looking at the variables properties table under section 1.1, where the min value for `appCat.builtin` is highly negative.

In [ ]:
# List of variables denoting time durations
time_variables = [var for var in df['variable'].unique() if 'appCat.' in var or var == 'screen']

# Remove records with negative durations for time variables
print("Removed records due to negative values:")
initial_count = df.shape[0]
for var in time_variables:
    before_count = df[df['variable'] == var].shape[0]
    df = df[~((df['variable'] == var) & (df['value'] < 0))]
    after_count = df[df['variable'] == var].shape[0]
    print(f"{before_count - after_count} from {var}")

total_removed = initial_count - df.shape[0]
print(f"Total records removed: {total_removed}")

#### Removing extreme values (Todo, Ryan)

For numeric variables that are not self-reported by the participants (so not mood, arousal, valence & activity, and also not call or SMS) we remove values that lie outside of the z-score threshold of ±3. Points that are more than 3 standard deviations away from the mean are considered outliers and should thus not be considered in our dataset. This should offer a more robust removal of outliers than other statistical methods like Interquartile Range, because it can better handle data distributions with heavier tails, like some of our variable values are as can be seen by the histograms for section 1.1.

In [ ]:
# Variables for which we remove outliers
exclude_vars = ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'call', 'sms']
outlier_vars = [var for var in df['variable'].unique() if var not in exclude_vars]

# Removing outliers using Z-score method
initial_count = df.shape[0]
for var in outlier_vars:
    # Calculate Z-scores for the variable
    var_data = df[df['variable'] == var]['value']
    z_scores = np.abs((var_data - var_data.mean()) / var_data.std())
    
    # Filter out data points where the absolute Z-score is greater than 3
    before_count = df[df['variable'] == var].shape[0]
    df = df[~((df['variable'] == var) & (z_scores > 3))]
    after_count = df[df['variable'] == var].shape[0]
    print(f'Removed {before_count - after_count} outliers from {var}')

total_removed = initial_count - df.shape[0]
print(f'Total records removed: {total_removed}')

#### Distribution of variable values post cleaning

In [ ]:
variables = [var for var in df['variable'].unique() if var not in ['sms', 'call']]
fig, axes = plt.subplots(nrows=len(variables), ncols=1, figsize=(10, 6 * len(variables)))

for ax, var in zip(axes.flatten(), variables):
    var_data = df[df['variable'] == var]['value'].dropna()
    mean = var_data.mean()
    std = var_data.std()
    # Adjust bins for better visualization based on data range and characteristics
    bins = min(30, int(var_data.nunique()))  # Use a minimum of 30 bins or less if fewer unique values

    ax.hist(var_data, bins=bins, alpha=0.75, color='blue', edgecolor='black', label=f'{var} Scores')
    ax.set_title(f'Distribution of {var} values')
    ax.set_xlabel(f'{var.capitalize()} Score')
    ax.set_ylabel('Frequency')
    ax.grid(axis='y', alpha=0.75)
    legend_label = f"Mean: {mean:.2f}, Std: {std:.2f}"
    ax.legend([f"{var.capitalize()} Scores\n{legend_label}"], loc='upper right', title='Statistics', frameon=True)

plt.tight_layout()
plt.show()

We can see that its much better......todo

#### Imputing missing values (Todo, Ryan)

#### Start on basic predicition

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# NOTE: this is still little basic format for the model, will maybe have to run on Google colab?


x = df # input features
y = df[df['variable'] == 'mood'] # target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Transform data 
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

x_train_reshaped = x_train_scaled.reshape(x_train_scaled.shape[0], x_train_scaled.shape[1], 1)
x_test_reshaped = x_test_scaled.reshape(x_test_scaled.shape[0], x_test_scaled.shape[1], 1)

# Define the CNN model, three layers
model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(x_train_reshaped.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(x_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(x_test_reshaped, y_test))

loss = model.evaluate(x_test_reshaped, y_test)
print(f'Test loss: {loss}')

predictions = model.predict(x_test_reshaped)

ValueError: Found input variables with inconsistent numbers of samples: [374651, 5641]

# Old code (I think Lieve's :))

In [ ]:
# clean data by dropping NA values

cleaned_data = df[df['value'] != 'NA'].dropna()

# TODO: remove outliers for each variable and write in report why/how

print(cleaned_data)

In [ ]:
# Histogram of mood 
plt.figure(figsize=(8, 6))
plt.hist(cleaned_data[cleaned_data['variable'] == 'mood']['value'], bins=10, color='skyblue', edgecolor='black')
plt.title('Histogram of Mood')
plt.xlabel('Mood')
plt.ylabel('Frequency')
plt.grid(True)
plt.tight_layout()
plt.show()


# Boxplot of arousal 
plt.figure(figsize=(8, 6))
plt.boxplot(cleaned_data[cleaned_data['variable'] == 'circumplex.arousal']['value'], vert=True)
plt.title('Boxplot of arousal')
plt.grid(True)
plt.tight_layout()
plt.show()

# Boxplot of valence 
plt.figure(figsize=(8, 6))
plt.boxplot(cleaned_data[cleaned_data['variable'] == 'circumplex.valence']['value'], vert=True)
plt.title('Boxplot of valence')
plt.grid(True)
plt.tight_layout()
plt.show()

# Time series plot of mood
# TODO: fix errors and readability of plot
grouped_data = cleaned_data.groupby('id')

# Plot for each 'id'
plt.figure(figsize=(30, 6))
for name, group in grouped_data:
    plt.plot(group[cleaned_data['variable'] == 'mood']['time'], group[cleaned_data['variable'] == 'mood']['value'], label=name, linewidth=1)
    # plt.plot(group['time'], group['value'], label=name, linewidth=1)
    # plt.plot(cleaned_data[cleaned_data['variable'] == 'mood']['time'], cleaned_data[cleaned_data['variable'] == 'mood']['value'], linewidth=1)

plt.title('Time Series Plot of Mood')
plt.xlabel('Time')
plt.ylabel('Mood')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# Time series of activity
# TODO: fix errors and readability of plot
grouped_data = cleaned_data.groupby('id')

# Plot for each 'id'
plt.figure(figsize=(30, 6))
for name, group in grouped_data:
    plt.plot(group[cleaned_data['variable'] == 'activity']['time'], group[cleaned_data['variable'] == 'activity']['value'], label=name, linewidth=1)
    # plt.plot(group['time'], group['value'], label=name, linewidth=1)
    # plt.plot(cleaned_data[cleaned_data['variable'] == 'mood']['time'], cleaned_data[cleaned_data['variable'] == 'mood']['value'], linewidth=1)

plt.title('Time Series Plot of Activity')
plt.xlabel('Time')
plt.ylabel('Activity')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Time series of activity AND mood

avg_activity = cleaned_data[cleaned_data['variable'] == 'activity'].groupby(cleaned_data['time'].dt.date)['value'].mean()
avg_mood = cleaned_data[cleaned_data['variable'] == 'mood'].groupby(cleaned_data['time'].dt.date)['value'].mean()

fig, ax1 = plt.subplots(figsize=(15, 6))

ax1.set_xlabel('Time')
ax1.set_ylabel('Activity', color='tab:red')
ax1.plot(avg_activity.index, avg_activity.values, color='tab:red')
ax1.tick_params(axis='y', labelcolor='tab:red')

ax2 = ax1.twinx() 
ax2.set_ylabel('Mood', color='tab:blue')
ax2.plot(avg_mood.index, avg_mood.values, color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

fig.tight_layout() 
plt.show()

# Time series of screen AND mood
avg_screen = cleaned_data[cleaned_data['variable'] == 'screen'].groupby(cleaned_data['time'].dt.date)['value'].mean()

fig, ax1 = plt.subplots(figsize=(15, 6))

ax1.set_xlabel('Time')
ax1.set_ylabel('screen', color='tab:red')
ax1.plot(avg_screen.index, avg_screen.values, color='tab:red')
ax1.tick_params(axis='y', labelcolor='tab:red')

ax2 = ax1.twinx() 
ax2.set_ylabel('Mood', color='tab:blue')
ax2.plot(avg_mood.index, avg_mood.values, color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

fig.tight_layout() 
plt.show()

# Time series of utilities app and mood
avg_ut = cleaned_data[cleaned_data['variable'] == 'appCat.utilities'].groupby(cleaned_data['time'].dt.date)['value'].mean()

fig, ax1 = plt.subplots(figsize=(15, 6))

ax1.set_xlabel('Time')
ax1.set_ylabel('Utilities', color='tab:red')
ax1.plot(avg_ut.index, avg_ut.values, color='tab:red')
ax1.tick_params(axis='y', labelcolor='tab:red')

ax2 = ax1.twinx() 
ax2.set_ylabel('Mood', color='tab:blue')
ax2.plot(avg_mood.index, avg_mood.values, color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

fig.tight_layout() 
plt.show()

In [ ]:
# convert to pandas dataframe
df = pd.DataFrame(cleaned_data, columns=["", "id", "time", "variable", "value"])

groups = df.groupby('variable')['value']

# TODO: fix statistical test so we can remove variables which are not significant

# Perform statistical tests for each variable
for var, group in groups:
    if var != 'mood':
        if len(group.unique()) > 2: 
            f_statistic, p_value = stats.f_oneway(*[group for name, group in groups])
            if p_value < 0.05:
                print(f"Variable '{var}' is significant (p-value: {p_value:.4f}, f stat: {f_statistic})")
            else:
                print(f"Variable '{var}' is not significant (p-value: {p_value:.4f}, f stat: {f_statistic})")